# 1910456 Mir Shafayat Ahmed
## CSC470 Introduction To Parallel Programming
### Summer 2023 Assignment


<a href="https://colab.research.google.com/github/shafayat1004/Parallel-Programming-Assignments/blob/main/1910456_MirShafayatAhmed_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/shafayat1004/Parallel-Programming-Assignments.git
%pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
!nvidia-smi

Cloning into 'Parallel-Programming-Assignments'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 151 (delta 10), reused 151 (delta 10), pack-reused 0
Receiving objects: 100% (151/151), 1.50 MiB | 12.13 MiB/s, done.
Resolving deltas: 100% (10/10), done.
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-qi2sqrvi
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-qi2sqrvi
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=8c2f36ea869580bb7698e8640b9538a9b9a2e5a380816deb5811b1a48c281727
  Stored in directory: /tmp/pip-ephem-wheel-cache-6j97_77p/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067

In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
def build_and_run (assignment_no: int, no_of_test_cases: int):
    !nvcc mp{assignment_no}.cu -o mp{assignment_no}

    print(f"Running MP{assignment_no}")
    print("\n======================================================================")

    if no_of_test_cases > 0:
        for i in range(no_of_test_cases):
            print(f"Test Case {i}:")
            print("----------------")
            !./mp{assignment_no} /content/Parallel-Programming-Assignments/MP{assignment_no}/data/{i}/*
    else:
        !./mp{assignment_no}
    
    print("======================================================================")

## MP0

In [ ]:
%%writefile mp0.cu
#include "/content/Parallel-Programming-Assignments/include/wb.h"

//@@ The purpose of this code is to become familiar with the submission
//@@ process. Do not worry if you do not understand all the details of
//@@ the code.

int main(int argc, char **argv) {
    int deviceCount;

    wbArg_read(argc, argv);

    cudaGetDeviceCount(&deviceCount);

    wbTime_start(GPU, "Getting GPU Data."); //@@ start a timer

    for (int dev = 0; dev < deviceCount; dev++) {
        cudaDeviceProp deviceProp;

        cudaGetDeviceProperties(&deviceProp, dev);

        if (dev == 0) {
            if (deviceProp.major == 9999 && deviceProp.minor == 9999) {
                wbLog(TRACE, "No CUDA GPU has been detected");
                return -1;
            } else if (deviceCount == 1) {
                //@@ WbLog is a provided logging API (similar to Log4J).
                //@@ The logging function wbLog takes a level which is either
                //@@ OFF, FATAL, ERROR, WARN, INFO, DEBUG, or TRACE and a
                //@@ message to be printed.
                wbLog(TRACE, "There is 1 device supporting CUDA");
            } else {
                wbLog(TRACE, "There are ", deviceCount,
                    " devices supporting CUDA");
            }
        }

        wbLog(TRACE, "Device ", dev, " name: ", deviceProp.name);
        wbLog(TRACE, " Computational Capabilities: ", deviceProp.major, ".",
            deviceProp.minor);
        wbLog(TRACE, " Maximum global memory size: ",
            deviceProp.totalGlobalMem);
        wbLog(TRACE, " Maximum constant memory size: ",
            deviceProp.totalConstMem);
        wbLog(TRACE, " Maximum shared memory size per block: ",
            deviceProp.sharedMemPerBlock);
        wbLog(TRACE, " Maximum block dimensions: ",
            deviceProp.maxThreadsDim[0], " x ", deviceProp.maxThreadsDim[1],
            " x ", deviceProp.maxThreadsDim[2]);
        wbLog(TRACE, " Maximum grid dimensions: ", deviceProp.maxGridSize[0],
            " x ", deviceProp.maxGridSize[1], " x ",
            deviceProp.maxGridSize[2]);
        wbLog(TRACE, " Warp size: ", deviceProp.warpSize);
    }

    wbTime_stop(GPU, "Getting GPU Data."); //@@ stop the timer

    return 0;
}

In [ ]:
assignment = 0

!nvcc mp{assignment}.cu -o mp{assignment}
print(f"Running MP{assignment}")
!./mp{assignment}

## MP1

In [19]:
%%writefile mp1.cu
#include "/content/Parallel-Programming-Assignments/include/wb.h"

__global__ void vecAdd(float *in1, float *in2, float *out, int len) {
	int idx = blockIdx.x*blockDim.x + threadIdx.x;
	if (idx < len)
		out[idx] = in1[idx] + in2[idx];
}

int main(int argc, char **argv) {
	wbArg_t args;
	int inputLength;
	float *hostInput1;
	float *hostInput2;
	float *hostOutput;
	float *deviceInput1;
	float *deviceInput2;
	float *deviceOutput;

	args = wbArg_read(argc, argv);

	wbTime_start(Generic, "Importing data and creating memory on host");
	hostInput1 = (float *)wbImport(wbArg_getInputFile(args, 0), &inputLength);
	hostInput2 = (float *)wbImport(wbArg_getInputFile(args, 1), &inputLength);
	hostOutput = (float *)malloc(inputLength * sizeof(float));
	wbTime_stop(Generic, "Importing data and creating memory on host");

	wbLog(TRACE, "The input length is ", inputLength);

	int size = inputLength * sizeof(float);

	wbTime_start(GPU, "Allocating GPU memory.");
	cudaMalloc((void **)&deviceInput1, size);
	cudaMalloc((void **)&deviceInput2, size);
	cudaMalloc((void **)&deviceOutput, size);
	wbTime_stop(GPU, "Allocating GPU memory.");

	wbTime_start(GPU, "Copying input memory to the GPU.");
	cudaMemcpy(deviceInput1, hostInput1, size, cudaMemcpyHostToDevice);
	cudaMemcpy(deviceInput2, hostInput2, size, cudaMemcpyHostToDevice);
	wbTime_stop(GPU, "Copying input memory to the GPU.");

	dim3 DimGrid((inputLength - 1) / 256 + 1, 1, 1);
	dim3 DimBlock(256, 1, 1);

	wbTime_start(Compute, "Performing CUDA computation");
	vecAdd << <DimGrid, DimBlock >> >(deviceInput1, deviceInput2, deviceOutput, inputLength);

	cudaDeviceSynchronize();
	wbTime_stop(Compute, "Performing CUDA computation");

	wbTime_start(Copy, "Copying output memory to the CPU");
	cudaMemcpy(hostOutput, deviceOutput, size, cudaMemcpyDeviceToHost);
	wbTime_stop(Copy, "Copying output memory to the CPU");

	wbTime_start(GPU, "Freeing GPU Memory");
	cudaFree(deviceInput1);
	cudaFree(deviceInput2);
	cudaFree(deviceOutput);
	wbTime_stop(GPU, "Freeing GPU Memory");

	wbSolution(args, hostOutput, inputLength);

	free(hostInput1);
	free(hostInput2);
	free(hostOutput);

	return 0;
}


Overwriting mp1.cu


In [20]:
assignment = 1
!nvcc mp{assignment}.cu -o mp{assignment}

for i in range(10):
  print(f"Running MP{assignment}")
  print("\n======================================================================")
  print(f"Test Case {i}:")
  print("----------------")
  !./mp1 /content/Parallel-Programming-Assignments/MP1/data/{i}/*
  print("======================================================================")


Test Case 0:
----------------
[Generic] 0.000167936 Importing data and creating memory on host
Trace main::27 The input length is 64
[GPU    ] 0.000166144 Allocating GPU memory.
[GPU    ] 0.000039936 Copying input memory to the GPU.
[Compute] 0.000026112 Performing CUDA computation
[Copy   ] 0.000018944 Copying output memory to the CPU
[GPU    ] 0.000106752 Freeing GPU Memory
Solution is correct.

Test Case 1:
----------------
[Generic] 0.000237056 Importing data and creating memory on host
Trace main::27 The input length is 128
[GPU    ] 0.000142848 Allocating GPU memory.
[GPU    ] 0.000045056 Copying input memory to the GPU.
[Compute] 0.000029184 Performing CUDA computation
[Copy   ] 0.000020992 Copying output memory to the CPU
[GPU    ] 0.000108032 Freeing GPU Memory
Solution is correct.

Test Case 2:
----------------
[Generic] 0.000215040 Importing data and creating memory on host
Trace main::27 The input length is 56
[GPU    ] 0.000130048 Allocating GPU memory.
[GPU    ] 0.000040

## MP2

In [ ]:
%%writefile mp2.cu
#include "/content/Parallel-Programming-Assignments/include/wb.h"

#define wbCheck(stmt)                                                     \
  do {                                                                    \
    cudaError_t err = stmt;                                               \
    if (err != cudaSuccess) {                                             \
      wbLog(ERROR, "Failed to run stmt ", #stmt);                         \
      wbLog(ERROR, "Got CUDA error ...  ", cudaGetErrorString(err));      \
      return -1;                                                          \
    }                                                                     \
  } while (0)

// Compute C = A * B
__global__ void matrixMultiply(float *A, float *B, float *C, int numARows,
                               int numAColumns, int numBRows,
                               int numBColumns, int numCRows,
                               int numCColumns) {
  //@@ Insert code to implement matrix multiplication here
}

int main(int argc, char **argv) {
  wbArg_t args;
  float *hostA; // The A matrix
  float *hostB; // The B matrix
  float *hostC; // The output C matrix
  float *deviceA;
  float *deviceB;
  float *deviceC;
  int numARows;    // number of rows in the matrix A
  int numAColumns; // number of columns in the matrix A
  int numBRows;    // number of rows in the matrix B
  int numBColumns; // number of columns in the matrix B
  int numCRows;    // number of rows in the matrix C (you have to set this)
  int numCColumns; // number of columns in the matrix C (you have to set
                   // this)

  args = wbArg_read(argc, argv);

  wbTime_start(Generic, "Importing data and creating memory on host");
  hostA = (float *)wbImport(wbArg_getInputFile(args, 0), &numARows,
                            &numAColumns);
  hostB = (float *)wbImport(wbArg_getInputFile(args, 1), &numBRows,
                            &numBColumns);
  //@@ Set numCRows and numCColumns
  numCRows = 0;
  numCColumns = 0;
  //@@ Allocate the hostC matrix
  wbTime_stop(Generic, "Importing data and creating memory on host");

  wbLog(TRACE, "The dimensions of A are ", numARows, " x ", numAColumns);
  wbLog(TRACE, "The dimensions of B are ", numBRows, " x ", numBColumns);

  wbTime_start(GPU, "Allocating GPU memory.");
  //@@ Allocate GPU memory here

  wbTime_stop(GPU, "Allocating GPU memory.");

  wbTime_start(GPU, "Copying input memory to the GPU.");
  //@@ Copy memory to the GPU here

  wbTime_stop(GPU, "Copying input memory to the GPU.");

  //@@ Initialize the grid and block dimensions here

  wbTime_start(Compute, "Performing CUDA computation");
  //@@ Launch the GPU Kernel here

  cudaDeviceSynchronize();
  wbTime_stop(Compute, "Performing CUDA computation");

  wbTime_start(Copy, "Copying output memory to the CPU");
  //@@ Copy the GPU memory back to the CPU here

  wbTime_stop(Copy, "Copying output memory to the CPU");

  wbTime_start(GPU, "Freeing GPU Memory");
  //@@ Free the GPU memory here

  wbTime_stop(GPU, "Freeing GPU Memory");

  wbSolution(args, hostC, numCRows, numCColumns);

  free(hostA);
  free(hostB);
  free(hostC);

  return 0;
}


  wbTime_stop(GPU, "Freeing GPU Memory");

  wbSolution(args, hostC, numCRows, numCColumns);

  free(hostA);
  free(hostB);
  free(hostC);

  return 0;
}


In [ ]:
assignment = 2
!nvcc mp{assignment}.cu -o mp{assignment}

for i in range(10):
  print(f"Running MP{assignment}")
  print("\n======================================================================")
  print(f"Test Case {i}:")
  print("----------------")
  !./mp1 /content/Parallel-Programming-Assignments/MP1/data/{i}/*
  print("======================================================================")

## MP3

In [ ]:
%%writefile mp3.cu
#include "/content/Parallel-Programming-Assignments/include/wb.h"

#define wbCheck(stmt)                                                     \
  do {                                                                    \
    cudaError_t err = stmt;                                               \
    if (err != cudaSuccess) {                                             \
      wbLog(ERROR, "Failed to run stmt ", #stmt);                         \
      wbLog(ERROR, "Got CUDA error ...  ", cudaGetErrorString(err));      \
      return -1;                                                          \
    }                                                                     \
  } while (0)

// Compute C = A * B
__global__ void matrixMultiply(float *A, float *B, float *C, int numARows,
                               int numAColumns, int numBRows,
                               int numBColumns, int numCRows,
                               int numCColumns) {
  //@@ Insert code to implement matrix multiplication here
}

int main(int argc, char **argv) {
    wbArg_t args;
    float *hostA; // The A matrix
    float *hostB; // The B matrix
    float *hostC; // The output C matrix
    float *deviceA;
    float *deviceB;
    float *deviceC;
    int numARows;    // number of rows in the matrix A
    int numAColumns; // number of columns in the matrix A
    int numBRows;    // number of rows in the matrix B
    int numBColumns; // number of columns in the matrix B
    int numCRows;    // number of rows in the matrix C (you have to set this)
    int numCColumns; // number of columns in the matrix C (you have to set
                    // this)

    args = wbArg_read(argc, argv);

    wbTime_start(Generic, "Importing data and creating memory on host");
    hostA = (float *)wbImport(wbArg_getInputFile(args, 0), &numARows,
                                &numAColumns);
    hostB = (float *)wbImport(wbArg_getInputFile(args, 1), &numBRows,
                                &numBColumns);
    //@@ Set numCRows and numCColumns
    numCRows = 0;
    numCColumns = 0;
    //@@ Allocate the hostC matrix
    wbTime_stop(Generic, "Importing data and creating memory on host");

    wbLog(TRACE, "The dimensions of A are ", numARows, " x ", numAColumns);
    wbLog(TRACE, "The dimensions of B are ", numBRows, " x ", numBColumns);

    wbTime_start(GPU, "Allocating GPU memory.");
    //@@ Allocate GPU memory here

    wbTime_stop(GPU, "Allocating GPU memory.");

    wbTime_start(GPU, "Copying input memory to the GPU.");
    //@@ Copy memory to the GPU here

    wbTime_stop(GPU, "Copying input memory to the GPU.");

    //@@ Initialize the grid and block dimensions here

    wbTime_start(Compute, "Performing CUDA computation");
    //@@ Launch the GPU Kernel here

    cudaDeviceSynchronize();
    wbTime_stop(Compute, "Performing CUDA computation");

    wbTime_start(Copy, "Copying output memory to the CPU");
    //@@ Copy the GPU memory back to the CPU here

    wbTime_stop(Copy, "Copying output memory to the CPU");

    wbTime_start(GPU, "Freeing GPU Memory");
    //@@ Free the GPU memory here

    wbTime_stop(GPU, "Freeing GPU Memory");

    wbSolution(args, hostC, numCRows, numCColumns);

    free(hostA);
    free(hostB);
    free(hostC);

    return 0;
}


In [ ]:
assignment = 3
!nvcc mp{assignment}.cu -o mp{assignment}

for i in range(10):
  print(f"Running MP{assignment}")
  print("\n======================================================================")
  print(f"Test Case {i}:")
  print("----------------")
  !./mp1 /content/Parallel-Programming-Assignments/MP1/data/{i}/*
  print("======================================================================")

## MP4

In [ ]:
%%writefile mp4.cu
#include "/content/Parallel-Programming-Assignments/include/wb.h"

#define wbCheck(stmt)                                                     \
  do {                                                                    \
    cudaError_t err = stmt;                                               \
    if (err != cudaSuccess) {                                             \
      wbLog(ERROR, "CUDA error: ", cudaGetErrorString(err));              \
      wbLog(ERROR, "Failed to run stmt ", #stmt);                         \
      return -1;                                                          \
    }                                                                     \
  } while (0)

//@@ Define any useful program-wide constants here

//@@ Define constant memory for device kernel here

__global__ void conv3d(float *input, float *output, const int z_size,
                       const int y_size, const int x_size) {
  //@@ Insert kernel code here
}

int main(int argc, char *argv[]) {
    wbArg_t args;
    int z_size;
    int y_size;
    int x_size;
    int inputLength, kernelLength;
    float *hostInput;
    float *hostKernel;
    float *hostOutput;
    float *deviceInput;
    float *deviceOutput;

    args = wbArg_read(argc, argv);

    // Import data
    hostInput = (float *)wbImport(wbArg_getInputFile(args, 0), &inputLength);
    hostKernel =
        (float *)wbImport(wbArg_getInputFile(args, 1), &kernelLength);
    hostOutput = (float *)malloc(inputLength * sizeof(float));

    // First three elements are the input dimensions
    z_size = hostInput[0];
    y_size = hostInput[1];
    x_size = hostInput[2];
    wbLog(TRACE, "The input size is ", z_size, "x", y_size, "x", x_size);
    assert(z_size * y_size * x_size == inputLength - 3);
    assert(kernelLength == 27);

    wbTime_start(GPU, "Doing GPU Computation (memory + compute)");

    wbTime_start(GPU, "Doing GPU memory allocation");
    //@@ Allocate GPU memory here
    // Recall that inputLength is 3 elements longer than the input data
    // because the first  three elements were the dimensions
    wbTime_stop(GPU, "Doing GPU memory allocation");

    wbTime_start(Copy, "Copying data to the GPU");
    //@@ Copy input and kernel to GPU here
    // Recall that the first three elements of hostInput are dimensions and
    // do
    // not need to be copied to the gpu
    wbTime_stop(Copy, "Copying data to the GPU");

    wbTime_start(Compute, "Doing the computation on the GPU");
    //@@ Initialize grid and block dimensions here

    //@@ Launch the GPU kernel here
    cudaDeviceSynchronize();
    wbTime_stop(Compute, "Doing the computation on the GPU");

    wbTime_start(Copy, "Copying data from the GPU");
    //@@ Copy the device memory back to the host here
    // Recall that the first three elements of the output are the dimensions
    // and should not be set here (they are set below)
    wbTime_stop(Copy, "Copying data from the GPU");

    wbTime_stop(GPU, "Doing GPU Computation (memory + compute)");

    // Set the output dimensions for correctness checking
    hostOutput[0] = z_size;
    hostOutput[1] = y_size;
    hostOutput[2] = x_size;
    wbSolution(args, hostOutput, inputLength);

    // Free device memory
    cudaFree(deviceInput);
    cudaFree(deviceOutput);

    // Free host memory
    free(hostInput);
    free(hostOutput);
    return 0;
}
